## Project Assignment - DA&DR with Python
### Vincent Haunberger (103598)

### Verwendete Pakete (abseits von im Kurs besprochenen)
- Json wird verwendet um mit Daten im Json-Format umzugehen
- Time wird verwendet um mit Datetime-Objekten umgehen zu können
- urlopen, certifiy und ssl werden für den Api-Handler verwendet; genauer um die HTTP requests ausführen zu können
- Für den Web-Scraper auf Yahoo werden die Pakete requests und bs4 benötigt
- Für den OLS-Trend-Schätzer wir das Paket sklearn verwendet; hier insebsondere die Klasse LinearRegression
- Um den Relative Strengh Index zu berechnen, wird das Modul pandas_ta verwendet
- Zusätzlich zu den gewönlichen Paketen für Dash wurde noch das Modul dash_bootstrap_components importiert, um ein schöneres UI zu erstellen

In [1]:
import json
import time
import datetime as dt
import pandas as pd

# Api requests
from urllib.request import urlopen
import certifi
import ssl

# Scraper
import requests
from bs4 import BeautifulSoup

# Linear regression
import numpy as np
from sklearn.linear_model import LinearRegression

# RSI
import pandas_ta as pta

# Dash
import dash
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dcc, html, Dash
from plotly.subplots import make_subplots
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

### API Handler
Diese Klasse beinhaltet alle Methoden, um Daten von der API financialmodelingprep.com herunterladen zu können. Eine Klasse ist hier sinnvoll um den Api-Key und die Basis-URL öfters wiederverwenden zu können.
Folgende Methoden sind enthalten:
- _init__: Init Methode der Klasse
- _get_jsonparsed_data: Methode führt HTML request aus und gibt Daten im Json Format zurück
- list_dj_companies: Gibt eine List aller Unternehmen im Dow Jones zurück
- company_profile: Gibt grundlegende Infos (Mitarbeiterzahl, Sektor, ...) eines Unternehmens als Json zurück
- company_prices: Gibt den Aktien Preis eines Unternehmens der letzten 'timeseries' Tage zurück.
- company_eps: Gibt historische Earnings eines Unternehmens zurück
- index_prices: Gibt den Index-Preis eines Index der letzten 'timeseries' Tage zurück.

Weitere Erklärungen zu einzelnen Zeilen finden sich zudem noch als Kommentar im Code selbst.

In [2]:
# API handle class
class DataApi():
    
    def __init__(self, keys):
        self.fmp_key = "apikey=" + keys["fmp"]
        self.fmp_url = "https://financialmodelingprep.com/api/v3/"
    
    
    def _get_jsonparsed_data(self, url):
        """
        Receive the content of ``url``, parse it as JSON and return the object.
        input: url (str)
        output: data (dict)
        """
        ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1)
        response = urlopen(url, context=ssl_context)
        data = response.read().decode("utf-8")
        time.sleep(0.5) # Verhinden von zu vielen Requests
        return json.loads(data)
    
    
    def list_dj_companies(self, output_type="pd"):
        url = self.fmp_url + "dowjones_constituent?" + self.fmp_key
        request = self._get_jsonparsed_data(url)
        if output_type=="pd":
            df = pd.DataFrame.from_dict(request)
            return df
        else:
            return request
    
    
    def company_profile(self, ticker):
        url = self.fmp_url + "profile/" + ticker + "?" + self.fmp_key
        return self._get_jsonparsed_data(url)
    
    
    def company_prices(self, ticker, timeseries, output_type="pd"):
        ''' timeseries: days to date '''
        url = self.fmp_url + "historical-price-full/" + ticker + "?timeseries=" + str(timeseries) + "&" + self.fmp_key
        request = self._get_jsonparsed_data(url)
        
        if output_type=="pd":
            df = pd.DataFrame.from_dict(request["historical"])
            df["symbol"] = ticker
            df['date'] = df['date'].apply(pd.to_datetime) # Umwandlung des Date-strings in datetime-Objekt
            df.set_index('date', inplace=True) # Date als Index
            df = df.sort_index()
            return df
        else:
            return request
    
    
    def company_eps(self, ticker, output_type="pd"):
        url = self.fmp_url + "earnings-surprises/" + ticker + "?" + self.fmp_key
        request = self._get_jsonparsed_data(url)
        
        if output_type=="pd":
            df = pd.DataFrame.from_dict(request)
            df['date'] = df['date'].apply(pd.to_datetime) # Umwandlung des Date-strings in datetime-Objekt
            return df
        else:
            return request
    
    
    def index_prices(self, ticker, timeseries, output_type="pd"):
        url = self.fmp_url + "historical-price-full/" + ticker + "?timeseries=" + str(timeseries) + "&"+ self.fmp_key
        request = self._get_jsonparsed_data(url)
        
        if output_type=="pd":
            df = pd.DataFrame.from_dict(request["historical"])
            df["symbol"] = ticker
            df['date'] = df['date'].apply(pd.to_datetime) # Umwandlung des Date-strings in datetime-Objekt
            df.set_index('date', inplace=True) # Date als Index
            df = df.sort_index()
            return df
        else:
            return request


### Web Scraper (Yahoo)
Diese Funktion gibt die Earnings-Estimates eines Unternehmens zurück. Die Daten werden von Yahoo über einen Web-Scaper gezogen und dann in ein Pandas DataFrame umgewandelt. 

In [3]:
# Web Scraper for Analyst Estimates from Yahoo
def analyst_estimates(ticker):
    # Base URL
    url = f"https://finance.yahoo.com/quote/{ticker}/analysis/"
    # Headers für den HTTP Request
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    time.sleep(1) # Um den Server nicht zu überlasten wird nur jede Sekunde ein Request getätigt.
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0] # Die erste Tabelle auf der Webpage bezieht sich auf die Earnings
    df["symbol"] = ticker
    df.columns = ['earningsEstimate', 'currentQtr', 'nextQtr','currentYear', 'nextYear', 'symbol']
    return df

### OLS Trend Regression
Diese Funktion erstellt eine OLS Trendregression auf Basis der Daten bis einen Monat vor heute. Hier wird mit 21 Tagen gerechnet, da ein Monat im Schnitt 21 Börsentage hat. Die Funktion gibt die gefitteten Daten zurück.

In [4]:
# OLS Trend Regression: requires ordinal dates
def ols_trend(data, n=21):
    # Fit model with data up to 'n' days before today
    x = data[['date_ordinal']][0:len(data)-n]
    y = data[['adjClose']][0:len(data)-n]
    model = LinearRegression()
    model.fit(x, y)
    # Predict full df
    data['pred'] = model.predict(data[['date_ordinal']])
    return data 

**Setup der API:**

Verwenden Sie am besten meinen API-Key, da dieser die Premium Endpoints enthält und nicht auf 250 Request pro Tag beschränkt ist.

In [5]:
# Api setup: financialmodelingprep 
keys = {"fmp": "432683882e2e8cf4ec18d4d1abafcd59"}
Api = DataApi(keys)

In [6]:
# get Dow-Jones Companies
dj_companies = Api.list_dj_companies()

# Options für den Company Dropdown
dj_companies_options = [{'label': row['name'], 'value': row['symbol']} for index, row in dj_companies.iterrows()]

### Download der Daten
Die Folgenden Cells laden die Daten für das Dashboard aus dem Internet herunter. Diesen Schritt schon vor dem Starten des Servers durchzuführen, verschnellert das Dashboard sehr, da die API- und HTTP-Requests viel Lade-Zeit in Anspruch nehmen. Stattdessen wird dieser Task einmal, vor Start des Dash-Servers durchgeführt und dann nur jeweils eine gefilterte Ansicht der Daten im Dashboard angezeigt.

ticker_list enthält alle Ticker-Symbols der Unternehmen im Dow-Jones. Im folgenden wird über jeden Ticker geloopt und die jeweiligen Daten-Downloads getätigt und zu einem Datensatz zusammengefügt.

***Der Download aller Daten kann einige Minuten dauern***

In [7]:
# Liste alle Ticker-Symbols
ticker_list = dj_companies.symbol.tolist()

In [8]:
# Estimates Data
df_analyst_est = pd.DataFrame()
for company in ticker_list:
    df_analyst_est = df_analyst_est.append(analyst_estimates(company))

In [9]:
# Company Profile Data
df_company_profiles = {}
for company in ticker_list:
    df_company_profiles[company] = Api.company_profile(company)[0]

Hier wird die Anzahl der Trading-Tage seit Anfang letzten Jahres berechnet, welche dann an die Methoden company_prices & index_prices übergeben wird, um den Timeframe des Charts festzulegen.

In [10]:
# Number of Business Days till today from beginning of last year
start = dt.date( 2021, 1, 4 ) # Erster Arbeitstag in 2021
end = dt.date.today()
b_days = np.busday_count( start, end )

In [11]:
# Company Prices Data
df_company_prices = pd.DataFrame()
for company in ticker_list:
    comp = Api.company_prices(company, b_days)
    # Hier wird noch zusätzlich die Spalte 'sector' angehängt, um später den Sector Mean berechnen zu können
    comp['sector'] = df_company_profiles[company]['sector']
    df_company_prices = df_company_prices.append(comp)

In [12]:
# Historic Earnings Data
df_historic_eps = pd.DataFrame()
for company in ticker_list:
    df_historic_eps = df_historic_eps.append(Api.company_eps(company))

In [13]:
# Dow Jones Data
df_dj_prices = Api.index_prices("%5EDJI", timeseries=b_days)

**Berechnung des Sector-Mean**

Hier wird der tägliche Sector-Mean berechnet, indem der Prices-Datensatz nach Sektor und Tag gruppiert wurde.

In [14]:
# Sector Mean
df_company_prices['index_date'] = df_company_prices.index
secs = df_company_prices.groupby(['sector', 'index_date']).mean()
# secs.xs('Technology')

### Dash App

Im Folgenden ist das Layout des Dashboards festgelegt, die Callbackfunktionen werden im Anschluss erläutert. 
 
Zuerst wird die Dash App initiiert, und dabei ein externen Stylesheet geladen (über das Modul 'dash_bootstrap_components') um einheitliche Designelemente einfügen zu können. Hiermit sind alle Schriftarten, Buttons, Überschriften usw. in einem Einheitlichen Style ohne diese einzeln zu konfigurieren.

Zudem wird das Layout festgesetzt. Hierbei habe ich ebenfalls das Modul dash_bootstrap_components verwendet um meine Seite in einzelne Zeilen und Spalten unterteilen zu können.

In [15]:
# Create dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

In [16]:
# Create dash layout
app.layout = dbc.Container([
    
    # Data Storage
    dcc.Store(id='data_comp_profile'),
    dcc.Store(id='data_comp_eps_hist'),
    dcc.Store(id='data_comp_eps_est'),
    dcc.Store(id='data_comp_prices'),
    dcc.Store(id='data_dj_prices'),
    dcc.Store(id='data_sector_mean'),
    
    
    # Header
    html.Br(),
    dbc.Row([
        
        # Heading
        dbc.Col([
            html.H2('Financial Dashboard'),
            html.Div(id="company_name_h1")
            
        ],width=10),
        
        # Logo
        dbc.Col([
            
            html.Div(id="out_company_img") ,
            
        ],width=2),
    ]),
    
    
    # Body: Earnings (& Sidebar)
    html.Br(),
    dbc.Row([
        
        # Sidebar Left
        dbc.Col([
            
            # Company Dropdown
            html.Div([
                dcc.Dropdown(
                    id='company_dd',
                    placeholder="Select company ...",
                    options=dj_companies_options,
                    value="AAPL"
                ),
            ], style={
                'paddingTop':10
            }),
            html.Br(),
            
            html.Div(id='out_company_info')
                
        ],width=3,
            style={ # style of Sidebar
                'background-color':"#324547",
                'border-radius': 10,}
        ),
        
        
        # Graph Content
        dbc.Col([
            
            # Earnings
            html.Div([
                dcc.Graph(id='hist_graph'),
                html.Br(),
                dcc.Graph(id='est_graph')
            ]),           
                
        ],width=9),
    ]),
    html.Br(),
    
    # Body: Adj. Close Price
    dbc.Row([
        
        dbc.Col([
            html.Div([
            dcc.Graph(id="price_graph")
            ]),    
        ], width = 10),
        
        
        dbc.Col([
            html.Div([
                dbc.Label("Reference"),
                dbc.RadioItems(
                    options=[
                        {"label": "None", "value": "None"},
                        {"label": "Dow Jones", "value": "Dow Jones"},
                        {"label": "Sector Mean", "value": "Sector Mean"}],
                    value="Dow Jones",
                    id="index_option"),
                ], style={'paddingTop':100}),
            
            html.Br(),
            
            html.Div([
                dbc.Checklist(
                options=[
                    {"label": "Show trend", "value": "active"}],
                value=["active"],
                id="trend_option", inline=True),
            ]),
            
            
        ], width = 2)
        
    ]),
    html.Br(),
    
    # Body: RSI
    dbc.Row([
        
        dbc.Col([
            html.Div([
            dcc.Graph(id="rsi_graph")
            ]),  
        ], width = 10),
        
        
        dbc.Col([
            html.Div([
            dbc.Label("Overbought/oversold"),
            dbc.Checklist(
                options=[
                    {"label": "Show indicator", "value": "active"}],
                value=["active"], id="indicator_option", inline=True),
            ], style={'paddingTop':100}),
            
        ], width = 2),

    ]),
    html.Br(),
    html.Br(),
    html.Br(),
    html.Br(),
    
   
])

**Loading Data into Dash**

In diesem Callback werden die Daten über das DCC Storage in die Dash App geladen und in der Browser Session gespeichert. Damit läuft die App performant, auch mit größeren Datenmengen. Beim Ändern des Unternehmens über das Dropdown wird diese Callback Funktion getriggert und filtert die Datensätze nach der jeweiligen Firma. Die Daten werden dann als Output des Callbacks an das Data-Storage übergeben. (Siehe zB. dcc.Store(id='data_comp_profile') oben im Dash-Layout).

Dash kann hierbei jedoch nur Daten im Json Format speichern und nicht Pandas Objekte. Somit müssen alle Daten erst in json Format umgewandelt werden und dann beim Laden in den einzelnen Callbacks wieder von Json zu Pandas DataFrames umgewandelt werden.

In [17]:
# Loading Data Callback
@app.callback([
    Output('data_comp_profile', 'data'),
    Output('data_comp_eps_hist', 'data'),
    Output('data_comp_eps_est', 'data'),
    Output('data_comp_prices', 'data'),
    Output('data_dj_prices', 'data'),
    Output('data_sector_mean', 'data')
], Input('company_dd', 'value'))
def get_data(company):
    company_profile = df_company_profiles[company]
    eps_hist = df_historic_eps[df_historic_eps.symbol == company].to_json(date_format='iso', orient='split')
    eps_est = df_analyst_est[df_analyst_est.symbol == company].to_json(date_format='iso', orient='split')
    comp_prices = df_company_prices[df_company_prices.symbol == company].to_json(date_format='iso', orient='split')
    dj_prices = df_dj_prices.to_json(date_format='iso', orient='split')
    sector_mean = secs.xs(company_profile['sector']).to_json(date_format='iso', orient='split')
    
    return company_profile, eps_hist, eps_est, comp_prices, dj_prices, sector_mean

**Company Info**

Dieses Callback gibt die Informationen in der Sidebar links, das Firmen-Logo oben rechts und den Firmennamen in der Überschrift zurück. Dabei nimmt es als Input die company_profile Daten, die jedoch schon vom get_data() Callback nach Unternehmen gefiltert sind.

In [18]:
# Company info & logo
@app.callback([
    Output('out_company_info', 'children'),
    Output('out_company_img', 'children'),
    Output('company_name_h1', 'children')],
    Input('data_comp_profile', 'data')
)
def get_company_info(info):
    div1 = html.Div([
        html.P("Stock Ticker: " + info["symbol"], style={"font-weight": "bold"}),
        html.P("Address: " + info["address"] + ", " + info["city"] + "\n" + info["state"]),
        html.P("CEO: " + info["ceo"]),
        html.P("Sector: " + info["sector"]),
        html.P("Industry: " + info["industry"]),
        html.P("Number of Employees: " + str(info["fullTimeEmployees"])),
        html.P("IPO-Date: " + info["ipoDate"]),
    ], style={"color":"white"})
    
    div2 = html.Div(
        html.Img(src=info["image"],
                 style={'height':'40%', 'width':'40%'}))
    
    div3 = html.Div(
        html.H1(info["companyName"], id='company-header'),
    )
    
    return div1, div2, div3

**Earnings Graph**

In diesem Callback werden 2 Grafiken zurückgegeben:
- Historische Earnings
- Estimated Earnings

Erklärungen zu einzelnen Code-Zeilen finden sich im Code als Kommentare.

In [19]:
# Earnings
@app.callback([
    Output('hist_graph', 'figure'),
    Output('est_graph', 'figure'),
    ],[
    Input('data_comp_eps_hist', 'data'),
    Input('data_comp_eps_est', 'data'),
])
def generate_eps_graphs(eps_hist, eps_est, n_periods=4):
    # Lade Daten (von Json zu Pandas df)
    hist = pd.read_json(eps_hist, orient='split')
    est = pd.read_json(eps_est, orient='split')
    
    est.set_index('earningsEstimate', inplace=True)
    
    # Erstllung eines Quarter-Tag im Format "Q1-2022"
    hist['quarter'] = hist['date'].dt.quarter # gibt Quartal als Integer zurück
    # Erstellt String ("x['date'][0:4]" schneidet hierbei die ersten 4 Zeichen (das Jahr) von dem Date-String ab)
    hist['quarter'] = hist.apply(lambda x: f"Q{x['quarter']}-{str(x['date'])[0:4]}", axis=1)
    
    # Date als Index setzten                             
    hist.set_index('date', inplace=True)

    # Letzen 4 Estimates
    hist = hist.sort_index()[len(hist)-n_periods-1:len(hist)-1]

    # Berechnet Squared Error und Mean Squared Error
    mse = (hist.estimatedEarning - hist.actualEarningResult) ** 2
    mse_text = str(round(mse.mean(), 5)) # Text für Legende
    mse_p = html.P("mse: " + mse_text)
    mse = mse.to_frame(name='mse').reset_index() # create df


    # HISTORIC ESTIMATES GRAPH
    hist_graph = make_subplots(specs=[[{"secondary_y": True}]])
                                 
    # Actuals
    hist_graph.add_trace(
        go.Bar(name='actual',
           x=hist.quarter,
           y=hist.actualEarningResult,
           marker_color="#3D8F99"),
        secondary_y=False)
                                 
    # Estimates
    hist_graph.add_trace(
        go.Bar(name='historic estimate',
           x=hist.quarter,
           y=hist.estimatedEarning,
           marker_color="#324547"),
        secondary_y=False)

    # Squared error
    hist_graph.add_trace(
        go.Scatter(name='squared error',
                   x=hist.quarter,
                   y=mse.mse,
                   marker_color="#FF4046"),
        secondary_y=True)
                                 
    # Title of axes
    hist_graph.update_yaxes(title_text="eps", secondary_y=False)
    hist_graph.update_yaxes(title_text="squared error", secondary_y=True)
                                 
    # Layout
    hist_graph.update_layout(
        title="Historic earnings per share (USD)",
        legend_title = "MSE: " + mse_text,
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linewidth=1,
            ticks='outside',
        ),
        yaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linewidth=1,
            ticks='outside',
        ), plot_bgcolor='#ffffff'
    )


    # ESTIMATED EARNINGS GRAPH
    est_graph = go.Figure([
        
        # Estimated AVG
        go.Bar(name="estimated (avg)",
            x=est.columns[:-1],
            y=est.loc["Avg. Estimate"],
            hovertext=["No. of Analysts: " + str(i) for i in est.loc["No. of Analysts"]],
            marker_color="#324547"),
        
        # Estimated HIGH
        go.Scatter(name="estimated (high)",
                  x=est.columns[:-1],
                  y=est.loc["High Estimate"],
                  marker_color="#00FF8C"),
        
        # Estimated LOW
        go.Scatter(name="estimated (low)",
                  x=est.columns[:-1],
                  y=est.loc["Low Estimate"],
                  marker_color="#FF4046")
    ])

    est_graph.update_yaxes(title_text="eps")

    est_graph.update_layout(
        title="Estimated earnings per share (USD)",
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linewidth=1,
            ticks='outside',
        ),
        yaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linewidth=1,
            ticks='outside',
        ), plot_bgcolor='#ffffff'
    )

    return hist_graph, est_graph,
    

**Prices Graph**

In diesem Callback wird 1 Grafiken zurückgegeben:
- Preis Grafik, mit der Option eine Trendlinie und den DJ-Index oder Sector-Mean als Vergleich einzublenden.

Erklärungen zu einzelnen Code-Zeilen finden sich im Code als Kommentare.

In [20]:
# Prices
@app.callback(
    Output('price_graph', 'figure'),[
    Input('data_comp_prices', 'data'),
    Input('data_dj_prices', 'data'),
    Input('index_option', 'value'),
    Input('company_dd', 'value'),
    Input('trend_option', 'value'),
    Input('data_sector_mean', 'data')
])
def generate_prices_graph(comp_prices, dj_prices, option1, company, show_trend, secs, n_periods=4):
    # Lade Daten (von Json zu Pandas df)
    prices = pd.read_json(comp_prices, orient='split')
    # Um eine OLS Regression durchführen zu können muss das Datum in ein ordinales Format umgewandelt werden
    prices['date_ordinal'] = prices.index.map(pd.Timestamp.toordinal)
    trend = ols_trend(prices)
    dj = pd.read_json(dj_prices, orient='split')
    sector_means = pd.read_json(secs, orient='split')
                                 
    # PRICE PLOT                          
    price_graph = make_subplots(specs=[[{"secondary_y": True}]])

    price_graph.add_trace(
            go.Scatter(name="Adj. Close Price: " + company,
                      x=prices.index,
                      y=prices.adjClose,
                      marker_color="#3D8F99",
                      line={"width":2}),
            secondary_y=False)
  
    # add dj or sector mean
    if option1 == "Dow Jones":
        price_graph.add_trace(
            go.Scatter(name=option1,
                      x=dj.index,
                      y=dj.adjClose,
                      marker_color="#00FF8C",
                      opacity=0.8),
            secondary_y=True)
    
    elif option1 == "Sector Mean":
        price_graph.add_trace(
            go.Scatter(name=option1,
                      x=sector_means.index,
                      y=sector_means.adjClose,
                      marker_color="#00FF8C",
                      opacity=0.8),
            secondary_y=True)
    
        
    if len(show_trend) > 0:
        price_graph.add_trace(
                go.Scatter(name="Trend line",
                          x=trend.index,
                          y=trend.pred,
                          marker_color="#FF4046",
                          opacity=0.8,
                          line={"dash": "dash"}),
                secondary_y=False)

    price_graph.update_yaxes(title_text="Adj. Close Price (USD)", secondary_y=False)
    price_graph.update_yaxes(title_text="Adj. Close Price " + option1, secondary_y=True)

    price_graph.update_layout(
        title="Adj. Close Price (USD): " + company,
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linewidth=1,
            ticks='outside',
        ),
        yaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linewidth=1,
            ticks='outside',
        ), plot_bgcolor='#ffffff'
    )


    return price_graph


**Relative Strenght Index Graph**

In diesem Callback wird 1 Grafiken zurückgegeben:
- RSI Grafik, mit der Option Linien einzufügen, welche anzeigen ob die Aktie überkauft oder überverkauft wurde. Diese Linien sind bei jeder Aktie bei der 70% und 30% Marke.

Erklärungen zu einzelnen Code-Zeilen finden sich im Code als Kommentare.

In [21]:
# RSI
@app.callback(
    Output('rsi_graph', 'figure'),[
    Input('data_comp_prices', 'data'),
    Input('indicator_option', 'value')
])
def generate_rsi_graph(comp_prices, option2, lenght=14):
    # Lade Daten (von Json zu Pandas DataFrame)
    prices = pd.read_json(comp_prices, orient='split')
    rsi = pta.rsi(prices['adjClose'], length = lenght)
    
    rsi_graph = go.Figure()

    rsi_graph.add_trace(
        go.Scatter(x=prices.index,
                   y=rsi,
                   name='RSI', line={'color': '#324547'}))
    
    if len(option2) > 0:
        #if option2[0] == "active":
        rsi_graph.add_trace(
            go.Scatter(x=prices.index,
                      y=[70] * len(prices),
                      name='70%', line={'color':'#FF4046', 'width':1}))

        rsi_graph.add_trace(
            go.Scatter(x=prices.index,
                      y=[30] * len(prices),
                      name='30%', line={'color':'#00FF8C', 'width':1}))

    rsi_graph.update_yaxes(title_text="RSI")

    rsi_graph.update_layout(
            title="Relative Strength Index (RSI) ",
            xaxis=dict(
                showline=True,
                showgrid=False,
                showticklabels=True,
                linewidth=1,
                ticks='outside',
            ),
            yaxis=dict(
                showline=True,
                showgrid=False,
                showticklabels=True,
                linewidth=1,
                ticks='outside',
            ), plot_bgcolor='#ffffff'
    )
    
    return rsi_graph

In [22]:
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
